In [ ]:
using JuMP
using MadNLP
using ProgressMeter
using CairoMakie

In [ ]:
num_atoms = 15
num_bonds = num_atoms - 1
fps = 60
animation_fps = 60
animation_scale = 4
pref_dist = 1.
init_L = num_bonds*pref_dist
break_dist = 2*pref_dist
max_L = (num_bonds-1)*pref_dist + 1.6*break_dist
max_pull_perc = max_L / init_L - 1.
min_crack = 0.1*pref_dist
max_crack = max_L - (num_bonds-1)*min_crack
pot_min = -1.
diss_coeff = 0.3
time_pull = 4.
l2_dissipation = false # if false => Kelvin-Voigt
soft_max_alpha = 5
time_horizon = 6*time_pull
animation_width = 800
hidpi_scaling = 2
file_name = "line"
fontsize = 16
only_video = true; # if false generate a folder of snapshots for each frame

In [ ]:
function lennard_jones(dist_sq; pot_min=-1)
    # Lennard-jones of squared distances.
    # The factor cbrt(2) assures that the global minimum 
    # is attained at cur_dist == pref_dist
    q = pref_dist^2/(cbrt(2)*dist_sq)
    -4*pot_min*(q^6 - q^3)
#     q = pref_dist^2/(2*dist_sq)
#     -4*pot_min*(q^2 - q)
end
xs = range(0.72*pref_dist^2, 5*pref_dist^2, length=1000)
W(dsq) = lennard_jones(dsq, pot_min=pot_min)
lines(xs, W.(xs))

In [ ]:
function dirichlet_delta(t)
    if t <= 2*time_pull
        return max_pull_perc*init_L*sin(pi/2*t/time_pull)
    else
        return 0
    end
end

function dirichlet_delta_compress(t)
    if t <= 3*time_pull
        return max_pull_perc*init_L*sin(pi/2*t/time_pull)
    else
        return -max_pull_perc*init_L
    end
end

function dirichlet_delta_twice(t)
    if t <= 5*time_pull
        return max_pull_perc*init_L*sin(pi/2*t/time_pull)
    else
        return max_pull_perc*init_L
    end
end

g(t) = init_L + dirichlet_delta_twice(t)

xs = range(0, time_horizon, length=1000)
lines(xs, g.(xs), color=:blue)

Task:

Given an initial length of the rod compute the minimal configuration of the atoms.
(No dissipation is used here.)

We try several initial values:
- uniform distribution
- crack at bond i

In [ ]:
function plot_rod!(ax, delta_y; shift=0., color=:blue)
    y = [0.]
    for dy in delta_y
        push!(y, y[end]+dy)
    end
    
    scatter!(ax, y, shift*ones(length(y)), color=color, markersize=10)
end
fig = Figure()
ax = Axis(fig[1, 1], aspect=3)
plot_rod!(ax, pref_dist*ones(num_bonds))
plot_rod!(ax, pref_dist*ones(num_bonds), shift=0.1, color=:red)
fig

In [ ]:
L = 2*num_bonds*pref_dist
broken_bond = 2
max_crack = L - (num_bonds-1)*pref_dist
sigma = pref_dist/100

function get_start_delta_y(i; favor=-1, L=2*num_bonds*pref_dist)
    uniform_dist = L / num_bonds
    
    if favor == -1
        return uniform_dist
    end
    
    diff = pref_dist/10
    if i != favor
        return uniform_dist - diff/(num_bonds-1)
    else
        return uniform_dist + diff
    end
end

function get_noisy_start_delta_y(sigma)
    uniform_dist = L / num_bonds
    start_delta_y = [uniform_dist + sigma*(2*rand()-1) for _ in 1:num_bonds]
    new_L = sum(start_delta_y)
    (L/new_L)*start_delta_y
end

# delta_y_start = [get_start_delta_y(i, favor=broken_bond, L=L) for i in 1:num_bonds]
# delta_y_start = get_noisy_start_delta_y(sigma)
delta_y_start = [get_start_delta_y(i, favor=broken_bond, L=L) for i in 1:num_bonds]

rate_independent = Model(
    ()->MadNLP.Optimizer(
        print_level=MadNLP.WARN,
        # acceptable_tol=1e-8,
        # max_iter=1000
    )
)

@variable(
    rate_independent,
    0 <= delta_y[i=1:num_bonds] <= max_crack,
    start=delta_y_start[i]
)

@constraint(
    rate_independent, lenght_L,
    sum(delta_y[i] for i in 1:num_bonds) == L
)

register(rate_independent, :W, 1, lennard_jones, autodiff = true)
@NLobjective(
    rate_independent, Min, 
    sum(W(delta_y[i]^2) for i in 1:num_bonds)
)

optimize!(rate_independent)

empty!(ax)
plot_rod!(ax, delta_y_start)
plot_rod!(ax, value.(delta_y), shift=1, color=:red)
fig

Try a global minimization without starting value.
(Of course this is much harder to optimize.)

In [ ]:
L = 100*num_bonds*pref_dist
max_crack = L - (num_bonds-1)*pref_dist

rate_independent = Model(
    ()->MadNLP.Optimizer(
        print_level=MadNLP.WARN,
        # acceptable_tol=1e-8,
        # max_iter=1000
    )
)

# no start value
@variable(rate_independent, 0 <= delta_y[i=1:num_bonds] <= max_crack)

@constraint(
    rate_independent, lenght_L,
    sum(delta_y[i] for i in 1:num_bonds) == L
)

register(rate_independent, :W, 1, lennard_jones, autodiff = true)
@NLobjective(
    rate_independent, Min, 
    sum(W(delta_y[i]^2) for i in 1:num_bonds)
)

optimize!(rate_independent) # Does not find the correct global minimum!

empty!(ax)
plot_rod!(ax, value.(delta_y))
println(value.(delta_y))
fig

In [ ]:
step = 2

minmove = Model(
    ()->MadNLP.Optimizer(
        print_level=MadNLP.WARN,
        blas_num_threads=8,
        # acceptable_tol=1e-8,
        # max_iter=1000
    )
)

prev_L = init_L
L = g((step-1)/fps)
# uniform distribution of atoms at the start of the simulation
@NLparameter(
    minmove, 
    prev_delta_y[i=1:num_bonds] == prev_L / num_bonds
)
@NLexpression(
    minmove, prev_y[i=1:num_atoms],
    sum(prev_delta_y[j] for j in 1:i-1),
)

@NLparameter(minmove, max_elong_sq[i=1:num_bonds] == value(prev_delta_y[i])^2)
@NLparameter(minmove, broken[i=1:num_bonds] == 0.)

delta_L = L - prev_L
# initial guess: uniform expansion of all bonds
@variable(
    minmove, min_crack <= delta_y[i=1:num_bonds] <= max_crack,
    start=value(prev_delta_y[i]) + delta_L/num_bonds
)
@expression(minmove, y[i=1:num_atoms], sum(delta_y[j] for j in 1:i-1))

# Dirichlet boundary condition
@constraint(minmove, dirichlet_right, y[end] == L)

# Energy
@expression(minmove, dist_sq[i=1:num_bonds], delta_y[i]^2)
register(minmove, :W, 1, lennard_jones, autodiff = true)
@NLexpression(
    minmove, energy_expr,
    sum((1-broken[i])*W(dist_sq[i]) + broken[i]*(
        (
            W(dist_sq[i])*exp(soft_max_alpha*W(dist_sq[i])) +
            W(max_elong_sq[i])*exp(soft_max_alpha*W(max_elong_sq[i]))
        ) / (exp(soft_max_alpha*W(dist_sq[i])) + exp(soft_max_alpha*W(max_elong_sq[i])))
    ) for i in 1:num_bonds)
)

# Dissipation
if l2_dissipation
    @NLexpression(
        minmove, dissipation_expr,
        .5*sum((y[i] - prev_y[i])^2 for i in 2:num_atoms-1)
    )
else
    @NLexpression(
        minmove, dissipation_expr,
        .5*sum((delta_y[i] - prev_delta_y[i])^2 for i in 1:num_bonds)
    )
end

@NLobjective(minmove, Min, energy_expr + diss_coeff*fps*dissipation_expr)

optimize!(minmove)

value.(y)

In [ ]:
function dissipation(y, prev_y)
    num_atoms = length(y)
    .5*sum((y[i] - prev_y[i])^2 for i in 2:num_atoms-1)
end
dissipation(value.(y), value.(prev_y)), value(dissipation_expr)

In [ ]:
function soft_max(x, y; alpha=soft_max_alpha)
    (x*exp(alpha*x) + y*exp(alpha*y)) / (exp(alpha*x) + exp(alpha*y))
end

xs = range(-1.5*break_dist, 1.5*break_dist, length=1000)
ys = [max(x, 0) for x in xs]
soft_ys = [soft_max(x, 0) for x in xs]
fig = Figure()
ax = Axis(fig[1,1])
lines!(ax, xs, ys, color=:blue)
lines!(ax, xs, soft_ys, color=:red)
fig

In [ ]:
function energy(y, max_elong_sq)
    num_bonds = length(y)-1
    dist_sq = (y[2:end]-y[1:end-1]).^2
    res = 0.
    for i in 1:num_bonds
        if max_elong_sq[i] < break_dist^2
            res += W(dist_sq[i])
        else
            res += soft_max(W(dist_sq[i]), W(max_elong_sq[i]))
        end
    end
    res
end
energy(value.(y), value.(max_elong_sq)), value(energy_expr)

In [ ]:
function total(y, prev_y, max_elong_sq)
    energy(y, max_elong_sq) + diss_coeff*fps*dissipation(y, prev_y)
end

total(value.(y), value.(prev_y), value.(max_elong_sq)), objective_value(minmove)

In [ ]:
# sanity check
delta_y_comp = value.(prev_delta_y) .+ delta_L/num_bonds
y_comp = Vector{Float64}(undef, num_atoms)
y_comp[1] = 0.
for i in 2:num_atoms
    y_comp[i] = y_comp[i-1] + delta_y_comp[i-1]
end
y_comp
total(y_comp, value.(prev_y), value.(max_elong_sq))

In [ ]:
steps = [value.(prev_y), value.(y)]

In [ ]:
fig = Figure()
ax = Axis(
    fig[1, 1],
    aspect=3,
    limits=(-0.5, (1 + max_pull_perc)*init_L+0.5, -0.5, 0.5)
)
fig

In [ ]:
@showprogress "Computing minmoves..." for step in 3:fps*time_horizon
    # update memory variable
    y_last = steps[end]
    delta_y_last = y_last[2:end] - y_last[1:end-1]
    comp_max_elong_sq = delta_y_last.^2
    new_max_elong_sq = max.(value.(max_elong_sq), comp_max_elong_sq)
    set_value.(max_elong_sq, new_max_elong_sq)
    
    # check for broken links
    new_broken = new_max_elong_sq .> break_dist^2
    set_value.(broken, new_broken)
    
    # new -> old
    # note: delta_y is the variable not y
    set_value.(prev_delta_y, value.(delta_y_last))
    prev_L = L
    L = g((step-1)/fps)
    
    # update rod length
    set_normalized_rhs(dirichlet_right, L)

    # update start value
    delta_L = L - prev_L
    set_start_value.(delta_y, delta_y_last .+ delta_L / num_bonds)

    optimize!(minmove)

    push!(steps, value.(y))
end

In [ ]:
length(steps)

In [ ]:
value.(broken)

In [ ]:
max_elong_sq

In [ ]:
function animate_steps(
    steps;
    file_name=file_name,
    fps=animation_fps, width=animation_width, fontsize=11, scale=animation_scale, 
    save_snapshots=!only_video
)
    experiments_folder = "experiments"
    if !ispath(experiments_folder)
        mkdir(experiments_folder)
    end
    if save_snapshots
        snapshots_folder = "$experiments_folder/$file_name"
        if !ispath(snapshots_folder)
            mkdir(snapshots_folder)
        end
    end
    
    aspect = 4
    ax_aspect = 4
    height = width/aspect
    max_right = maximum([y[end] for y in steps])
    min_left = minimum([y[1] for y in steps])
    delta = 0.1
    max_len = max_right - min_left + 2delta
    ax_height = max_len/ax_aspect
    N = length(steps[1])
    
    fig = Figure(resolution=(scale*width, scale*height), fontsize=scale*fontsize)
    ax = Axis(
        fig[1, 1],
        limits=(min_left-delta, max_right+delta, -ax_height/2, ax_height/2),
        aspect=ax_aspect,
    )
    
    y = zeros(N)
    
    frame = 1
    record(fig, "experiments/$file_name.mp4", steps; framerate = fps) do (x)
        empty!(ax)
        scatter!(x, y, color=:blue, markersize=scale*6)
        if save_snapshots
            save("$snapshots_folder/$(file_name)_$frame.png", fig)
        end
        frame += 1
    end
end

In [ ]:
animate_steps(steps[1:4:end])

In [ ]:
function plot_energy(steps)
    ts = [1/fps*(i-1) for i in 1:length(steps)]
    es = Vector{Float64}()
    max_elong_sq = zeros(num_bonds)
    for y in steps
        push!(es, energy(y, max_elong_sq))
        # update maximal elongation
        dist_y_sq = (y[2:end] - y[1:end-1]).^2
        max_elong_sq = max.(max_elong_sq, dist_y_sq)
    end
    lines(ts, es)
end
plot_energy(steps)

In [ ]:
function plot_dissipation(steps)
    ts = [1/fps*(i-1) for i in 1:length(steps)]
    ds = [0.]
    prev_y = steps[1]
    for y in steps[2:end]
        push!(ds, diss_coeff*fps*dissipation(y, prev_y))
        prev_y = y
    end
    lines(ts, ds)
end

plot_dissipation(steps)

In [ ]:
function plot_total(steps)
    ts = [1/fps*(i-1) for i in 1:length(steps)]
    tots = Vector{Float64}()
    max_elong_sq = zeros(num_bonds)
    tots = [energy(steps[1], max_elong_sq)]
    prev_y = steps[1]
    for y in steps[2:end]
        # update maximal elongation
        dist_y_sq = (prev_y[2:end] - prev_y[1:end-1]).^2
        max_elong_sq = max.(max_elong_sq, dist_y_sq)
        push!(tots, energy(y, max_elong_sq) + diss_coeff*fps*dissipation(y, prev_y))
        prev_y = y
    end
    lines(ts, tots)
end

plot_total(steps)